# TP - Environnement Snake

Les entrainements d'agents de ce notebook peuvent être gourmands en puissance de calcul. Si votre machine est trop lente, basculez sur Colab.

Pour commencer, quelques cellules utilitaires pour une meilleure expérience *noteboook* :

In [1]:
#Uniquement utilse sous Colab
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !rm -rf ESGI-M2-IABD/
    !git clone https://github.com/pcouy/ESGI-M2-IABD
    !pip install --upgrade pip
    !pip install ESGI-M2-IABD/code

In [23]:
# Modification de la taille des sorties scrollables
# Rend également les sorties scrollables redimensionnables
from IPython.display import HTML, Image, display
HTML("""
<style>
.jp-CodeCell.jp-mod-outputsScrolled .jp-Cell-outputArea, div.output_scroll { max-height: 70vh; height:70vh; resize: vertical;}
</style>
""")

In [24]:
# Fonctions utilitaires pour afficher interactivement les vidéos des épisodes de test
# ATTENTION : Le slider de view_videos ne fonctionne que s'il n'y a pas d'exécution déjà en cours !
import os, io, glob, base64, threading
from ipywidgets import interact, widgets
from IPython.display import HTML, Image, display, Video

def load_and_display(path):
    """Show a video at `path` within IPython Notebook."""
    if not os.path.isfile(path):
        raise NameError("Cannot access: {}".format(path))

    video = io.open(path, "r+b").read()
    encoded = base64.b64encode(video)

    display(HTML(
        data="""
        <video alt="test" controls>
        <source src="data:video/mp4;base64,{0}" type="video/mp4"/>
        </video>
        """.format(encoded.decode("ascii"))
    ))

def show_last_video(agent_save_dir):
    dirname = os.path.join(agent_save_dir, "videos")
    files = sorted([f for f in os.listdir(dirname)])
    file = files[-1]
    path = os.path.join(dirname, file, "rl-video-episode-0.mp4")
    load_and_display(path)

def view_videos(agent):
    dirname = os.path.join(agent.save_dir, "videos")
    files = sorted([f for f in os.listdir(dirname)])
    print(files)
    N = len(files)
    def d(file: str) -> None:
        path = os.path.join(dirname, file, "rl-video-episode-0.mp4")
        load_and_display(path)
    interact(d, file=widgets.SelectionSlider(options=files, value=files[-1]))

## Agent tabulaire

La cellule ci-dessous donne un exemple d'entraînement d'agent tabulaire sur le jeu Snake. Lancez la cellule, et constatez l'apprentissage de l'agent avec les données de sorties (dossier `results`)

In [ ]:
import code_tp as TP
from code_tp import agents, value_functions, policies
from code_tp import wrappers
import gym
    
env = agents.snake.make_tabular_snake_env(4,5)
env = wrappers.utils.BoredomWrapper(env)
a=TP.create_agent_from_env(env,
agent_class=agents.base.QLearningAgent,
value_class=value_functions.tabular.TabularQValue,
policy_class=policies.greedy.EGreedyPolicy,
agent_args={
    'gamma':0.99
},
value_args={
    'lr':0.5, 'lr_decay':5e-7, 'lr_min':0.1,
    'default_value': 0
},
policy_args={
    'greedy_policy_class': policies.greedy.GreedyQPolicy,
    'epsilon': 1, 'epsilon_decay': 2e-4, 'epsilon_min': 0.05,
    'epsilon_test':0
})
a.train(2000, 200, test_callbacks=[show_last_video])

In [ ]:
a.plot_stats(save_dir=None)
view_videos(a)

Consultez la [documentation du TP](https://pcouy.github.io/ESGI-M2-IABD/index.html), et particulièrement la [documentation et le code de `agents.snake`](https://pcouy.github.io/ESGI-M2-IABD/code_tp/agents/snake.html) ainsi que [la classe `TabularObservation`](https://pcouy.github.io/ESGI-M2-IABD/code_tp/wrappers/utils.html#TabularObservation).

>Les trois classes concernées héritent toutes de la classe `gym.core.ObservationWrapper`. Trouvez le code de cette classe sur [le dépôt Github d'OpenAI Gym](https://github.com/openai/gym/). À quoi sert cette classe ?
>
>Pourquoi utiliser les classes `TabularObservation` et celles contenues dans `agent.snake` ? Que se serait-il passé si on avait appliqué l'algorithme de *Q-learning* tabulaire naïvement (*ie* sans utiliser ces *wrappers*) ?
>
>Testez l'agent tabulaire pour différentes valeurs des paramètres de `agents.snake.make_tabular_snake_env`.
>
>Que se passe-t-il lorsque la taille de la grille augmente ? Que se passe-t-il si le nombre de niveaux de discrétisation est trop élevé ? trop faible ?
>
>Quelle influence a le paramètre `default_value` ?

## Approximation linéaire de la fonction de valeur

>Consultez les documentations de `agents.snake.make_feature_snake_env` et `value_functions.linear` pour entrainer un agent utilisant une fonction de valeur approximée linéairement.
>
>Un taux d'apprentissage de 0.5 est-t-il toujours adapté ?
>
>Commentez les différences d'apprentissage avec l'agent linéaire (vitesse d'apprentissage, qualité de la stratégie, conséquences des changements de taille de grille)

In [19]:
# Instanciez les classes nécessaires et lancez l'entrainement ici

> Étudiez soigneusement [le code de la fonction de valeur linéaire](https://pcouy.github.io/ESGI-M2-IABD/code_tp/value_functions/linear.html). En quoi cette fonction de valeur s'apparente-t-elle déjà à un "réseau" de neurones contenant un unique neurone ?
> 
> Pouvez vous reconnaitre la ligne de code implémentant la descente de gradient dans la classe `LinearQValue` ?

## Initiation à PyTorch

Pour toutes les implémentations d'apprentissage **profond** (*ie* basées sur des réseaux de neurones) par renforcement de la suite du cours, nous utiliserons le [*framework* **PyTorch**](https://pytorch.org/). 

Commencez par installer la librairie :

```
pip install torch torchvision
```

Vous allez dans un premier temps suivre le [tutoriel d'initiation à PyTorch](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html). Il est vivement conseillé de sauvegarder les notebooks utilisés pour suivre le tutoriel (chaque chapitre du tutoriel est un notebook), afin de conserver une trace qui vous servira de référence pour la suite du cours.

Voici quelques conseils pour tirer profit du tutoriel au maximum :

* Utilisez les cellules *markdown* pour produire des documents qui se suffisent à eux-même. Utilisez les différents niveaux de titres pour vous retrouver facilement dans votre document à l'avenir.
* À toutes les étapes du tutoriel, n'hésitez pas à sortir du chemin tracé et à expérimenter vous même (sur les manipulations de tenseurs, sur le valeurs des hyperparamètres, différentes architectures de réseaux de neurones, etc) sur tout ce qui éveille votre curiosité. Conservez le code de ces expériences dans des cellules de vos *notebooks* et gardez une trace de ce que vous avez découvert (bonnes valeurs des hyperparamètres, piège à éviter sur certaines manipulations de tenseurs, etc)
* Appliquez vous à suivre toutes les étapes du tutoriel, même celles qui vous semblent triviales, et intégrez les dans votre *notebook* à la manière d'une "fiche de révision".
* Usez et abusez de [la documentation](https://pytorch.org/docs/stable/index.html).
* Le code de la dernière partie du tutoriel ("Training a Classifier") peut être lent à exécuter sur certaines machines. Ne pas hésiter à basculer sur [Google Colab](https://colab.research.google.com/) pour cette partie.
* Sollicitez moi autant que nécessaire, à l'oral pendant les séances, ou sur l'espace "Discussions" du *Github*, dès qu'un point vous semble obscure ou vous bloque.

Tous ces conseils, bien que chronophages, vous permettront de dompter au plus vite cet outil complexe. La documentation officielle est un outil précieux, mais votre familiarité avec vos propres notes vous feront gagner beaucoup de temps pour la suite.

Une bonne maitrise de *PyTorch* sera nécessaire pour suivre correctement les exercices pratiques de la suite du cours.

> Après avoir terminé le tutoriel, réimplémentez l'approximation linéaire de la fonction de valeur, en utilisant les outils de PyTorch : 
>
> * Vous remplacerez le tableau *Numpy* `self.weights` dans la classe `LinearQValue` par un module *PyTorch* (classe `Net` ci-dessous), *ie* un réseau de neurones. Ce réseau sera équivalent au réseau implémenté par l'approximation linéaire fournie (classe `LinearQValue`). ([Indice](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html))
> * Au lieu de faire "manuellement" la mise à jour des poids, vous utiliserez un [optimiseur SGD (pour Stochastic Gradient Descent)](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html#torch.optim.SGD). Vous conserverez tous les paramètres par défaut, à l'exception du taux d'apprentissage que vous ajusterez.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from value_functions.base import DiscreteQFunction

class Net(nn.Module):
    pass # REMPLIR ICI

class TorchLinearQValue(DiscreteQFunction):
     def __init__(self, env, default_value=0, *args, **kwargs):
        super().__init__(env, *args, **kwargs) # A CONSERVER
        # REMPLIR ICI

    def add_bias(self, state):
        pass # REMPLIR ICI

    def __call__(self, state, action):
        pass # REMPLIR ICI

    def from_state(self, state):
        pass # REMPLIR ICI
    
    def update(self, state, action, target_value):
        # REMPLIR ICI
        super().update(state, action, target_value) # A CONSERVER

Votre nouvelle fonction de valeur doit pouvoir remplacer "telle quelle" l'approximation linéaire en *Numpy* fournie (dans l'entrainement de l'agent ci-dessus, vous devez pouvoir remplacer `value_functions.linear.LinearQValue` par `TorchLinearQValue`).

> Instanciez et entrainez un agent utilisant la fonction de valeur que vous venez de définir, et confirmez son bon fonctionnement :

La fonction de valeur que vous venez d'implémenter (classe `TorchLinearQValue`) peut désormais vous servir de base pour créer des fonctions de valeur *neurales* arbitrairement complexes (attention cependant à la dimension des observations qui doit être égale à la dimension des *inputs* du réseau de neurone).

> Reprenez votre classe `Net` + le code d'instanciation et d'endtrainement ci-dessus. Copiez-collez les ci-dessous. Modifiez la classe Net pour expérimenter avec l'architecture du réseau de neurones. Avant de tester chaque modification, essayez de prédire ses conséquences sur l'apprentissage de l'agent.
>
> * Ajoutez une couche au réseau de neurones. Faites varier la taille de cette couche.
> * Si vous trouvez une bonne valeur pour la taille de cette nouvelle couche, essayez d'ajouter plus de couches de la même taille.
>
> Quelle sont les conséquences de ces modifications sur l'apprentissage de l'agent ? Attendiez-vous ce résultat ?